In [13]:
from IPython.display import Image, display
import PIL.Image
import io
import torch
import numpy as np
from processing_image import Preprocess
from visualizing_image import SingleImageViz
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils
import pandas as pd
import base64
from transformers import VisualBertForQuestionAnswering, BertTokenizerFast


In [2]:
URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/images/input.jpg"
OBJ_URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/genome/1600-400-20/objects_vocab.txt"
ATTR_URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/genome/1600-400-20/attributes_vocab.txt"
VQA_URL = "https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt"

ROOT = f".."
DATASET_PATH = f"{ROOT}/datasets/VQAv2.csv"


In [3]:
# for visualizing output
def showarray(a, fmt="jpeg"):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [4]:
# load object, attribute, and answer labels

objids = utils.get_data(OBJ_URL)
attrids = utils.get_data(ATTR_URL)
vqa_answers = utils.get_data(VQA_URL)

In [5]:
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)

image_preprocess = Preprocess(frcnn_cfg)

bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
visualbert_vqa = VisualBertForQuestionAnswering.from_pretrained("uclanlp/visualbert-vqa")

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /u/tb21/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [6]:
#load dataset

dataset_df = pd.read_csv(DATASET_PATH)
raw_images = dataset_df.image.to_list()

In [20]:
bytes_image = raw_images[0]
bytes_image

"b'...'"

In [24]:
# This portion is part of my test code

img=PIL.Image.open(io.BytesIO(base64.b64decode(bytes_image)))



UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fdb9031d470>

In [19]:

img=PIL.Image.open(io.BytesIO(base64.b64decode(raw_images[0])))


img

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7fdb9031d650>

In [9]:
# image viz
#frcnn_visualizer = SingleImageViz(URL, id2obj=objids, id2attr=attrids)
# run frcnn
images, sizes, scales_yx = image_preprocess(raw_images)
output_dict = frcnn(
    images,
    sizes,
    scales_yx=scales_yx,
    padding="max_detections",
    max_detections=frcnn_cfg.max_detections,
    return_tensors="pt",
)
# add boxes and labels to the image

# frcnn_visualizer.draw_boxes(
#     output_dict.get("boxes"),
#     output_dict.pop("obj_ids"),
#     output_dict.pop("obj_probs"),
#     output_dict.pop("attr_ids"),
#     output_dict.pop("attr_probs"),
# )
# showarray(frcnn_visualizer._get_buffer())

BGR
cannot identify image file <_io.BytesIO object at 0x7fdb928a4fb0>
b'...'


AssertionError: could not connect to: b'...'

In [17]:
test_questions_for_url1 = [
    "Where is this scene?",
    "what is the man riding?",
    "What is the man wearing?",
    "What is the color of the horse?"
]

# Very important that the boxes are normalized
#normalized_boxes = output_dict.get("normalized_boxes")
features = output_dict.get("roi_features")

In [18]:
for test_question in test_questions_for_url1:
    test_question = [test_question]

    inputs = bert_tokenizer(
        test_question,
        padding="max_length",
        max_length=20,
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    
    ###
    
    
    #labels = torch.tensor([[0.0, 1.0]]).unsqueeze(0)  # Batch size 1, Num labels 2  
    
    
    ###
    
    
    output_vqa = visualbert_vqa(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        visual_embeds=features,
        visual_attention_mask=torch.ones(features.shape[:-1]),
        token_type_ids=inputs.token_type_ids,
        output_attentions=False,
        
    )
    # get prediction
    pred_vqa = output_vqa["logits"].argmax(-1)
    
    
    loss = output_vqa.loss
    scores = output_vqa.logits
    
    
    print("Question:", test_question)
    print("prediction from VisualBert VQA:", vqa_answers[pred_vqa])
    print("loss: ", loss)
    print("scores: ", scores)
    

Question: ['Where is this scene?']
prediction from VisualBert VQA: outside
loss:  None
scores:  tensor([[-5.3181, -0.7488, -2.6537,  ..., -2.8860,  5.1213, -1.5156]],
       grad_fn=<ViewBackward0>)
Question: ['what is the man riding?']
prediction from VisualBert VQA: nothing
loss:  None
scores:  tensor([[-8.3704,  2.9947, -2.8991,  ..., -1.8537, -2.1171, -2.5161]],
       grad_fn=<ViewBackward0>)
Question: ['What is the man wearing?']
prediction from VisualBert VQA: shirt
loss:  None
scores:  tensor([[-7.2811,  2.0504, -2.6816,  ...,  0.5268, -1.3550, -1.8488]],
       grad_fn=<ViewBackward0>)
Question: ['What is the color of the horse?']
prediction from VisualBert VQA: brown
loss:  None
scores:  tensor([[-12.2276,   5.7673,  -2.5328,  ...,  -3.2615,  -1.5216,  -3.1597]],
       grad_fn=<ViewBackward0>)
